In [3]:
!pip install flair

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 20.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-no

In [4]:
from transformers import BertTokenizer, BertForSequenceClassification
from google.colab import drive
import torch

from nltk.tokenize import sent_tokenize
from transformers import pipeline

from flair.data import Sentence
from flair.models import SequenceTagger

import re

In [5]:
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# set environment as googledrive to folder "resource"
data_path =  "/resource/"

try:
    drive.mount('/content/drive')
    data_path = "/content/drive/My Drive/saved_model/"

except:
    print("You are not working in Colab at the moment :(")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
# Model ve tokenizer'ı yükle
loaded_model = BertForSequenceClassification.from_pretrained(data_path)
loaded_tokenizer = BertTokenizer.from_pretrained(data_path)

print("Model ve tokenizer başarıyla yüklendi.")

Model ve tokenizer başarıyla yüklendi.


In [8]:
# Modeli değerlendirme moduna al
loaded_model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
loaded_model.to(device)  # Move the model to the GPU

# Etiketlerin ve sayısal eşlemelerin tanımlanması
label_mapping = {
    0: "notr",
    1: "positive",
    2: "negative",
    3: "positive|negative",
}


In [9]:
ner_recognizer = SequenceTagger.load('flair/ner-english-large')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

2024-08-08 19:25:33,591 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


In [115]:
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk

# NLTK'nin gerekli verilerini indirin (ilk kullanımda gerekebilir)
nltk.download('punkt', quiet=True)

def split_sentences(text: str):
    sentences = sent_tokenize(text, language='turkish')
    return sentences

comment = "Turkcell ve Vodafone yaptıkları çalışmalarla Türkiye'de iletişimi kolaylaştırmaktadırlar ancak çok miktarda para talep etmektedirler."

sentences = split_sentences(comment)
print(comment)

Turkcell ve Vodafone yaptıkları çalışmalarla Türkiye'de iletişimi kolaylaştırmaktadırlar ancak çok miktarda para talep etmektedirler.


In [94]:
sentiment_analysis = pipeline("sentiment-analysis", model=data_path)

In [116]:
def remove_word(sentence, word):
    words = word_tokenize(sentence)
    filtered_words = [w for w in words if w != word]
    return " ".join(filtered_words)

def separate_sentences_via_conjunctions(sentence: str) -> list[str]:
    words: list[str] = word_tokenize(sentence)
    negative_conjunctions: list[str] = [
        "ama", "fakat", "lakin",
        "ancak", "oysa", "halbuki",
        "gerçi"
    ]

    split_sentences = []
    current_sentence = []

    for word in words:
        if word in negative_conjunctions:
            split_sentences.append(" ".join(current_sentence))
            current_sentence = []
        current_sentence.append(word)

    if current_sentence:
        split_sentences.append(" ".join(current_sentence))

    filtered_split_sentences: list[str] = []

    for _sentence in split_sentences:
        for conjunction in negative_conjunctions:
            if conjunction in _sentence:
                _sentence = remove_word(_sentence, conjunction)
        filtered_split_sentences.append(_sentence.strip())

    return filtered_split_sentences

In [118]:
from tkinter.constants import E
firm_list = []
dandan = []
results = []


def cut_after_apostroph(line):
    return re.sub(r"'.*$", "", line)

def sentiment_analyzer(sentence_i: str, firm: str):
    firm_list.append(firm)
    sentiment_result = sentiment_analysis(sentence_i)
    sentiment = sentiment_result[0]['label']

    if sentiment == "LABEL_1":
        results.append({"entity": firm, "sentiment": "olumlu"})
    elif sentiment == "LABEL_0":
        results.append({"entity": firm, "sentiment": "nötr"})
    elif sentiment == "LABEL_2":
        results.append({"entity": firm, "sentiment": "olumsuz"})
    elif sentiment == "LABEL_3":
        results.append({"entity": firm, "sentiment": "olumlu"})
        results.append({"entity": firm, "sentiment": "olumsuz"})

    return sentiment

def analyze_sentences(sentences):
    firm_list = []
    global inside_sentences
    pattern = r'@(\w+(\_\w+)*)'
    matches = re.findall(pattern, comment)
    result = [match[0] for match in matches]
    dandan.append(result)
    for i in dandan:
        for j in i:
            firm_list.append(j)
    for sentence in sentences:
        all_entities = []
        firms = []
        firmcount = 0
        text = Sentence(sentence)
        ner_recognizer.predict(text)

        for entity in text.get_spans('ner'):
            all_entities.append({
                "entity_name": entity.text,
                "type": entity.get_label('ner').value
            })

        # Firma olarak kategorize edilenleri çekiyoruz.
        for entity in all_entities:
            if entity['type'] == 'ORG':

                firm_list.append(cut_after_apostroph(entity['entity_name']))
                firmcount += 1
        # Tekrarlayan firma adlarının tek bir tanesini aldı.
        firm_list = list(set(firm_list))
        firmcount = len(firm_list)
        if firmcount == 1:
            for firm in firm_list:
                if firm in sentence:
                    sentiment = sentiment_analyzer(sentence, firm)
        else:
            inside_sentences = separate_sentences_via_conjunctions(sentence)
            if len(inside_sentences) == 1:
                if "," in inside_sentences[0]:
                    inside_sentences = inside_sentences[0].split(",")
                    for i in inside_sentences :
                        for firm in firm_list:
                            if firm in i:
                                sentiton = sentiment_analysis(inside_sentences[0])[0]['label']
                                if sentiton == "LABEL_0":
                                  print(inside_sentences)
                                  inside_sent = "".join(inside_sentences)
                                  sentiment = sentiment_analyzer(inside_sent, firm)
                                else:
                                  sentiment = sentiment_analyzer(i, firm)

                else:
                  inside_sentences = separate_sentences_via_conjunctions(sentence)
                  if len(inside_sentences) == 1:
                      for _sentence in inside_sentences:
                         for firm in firm_list:
                            if firm in _sentence:
                              sentiment = sentiment_analyzer(_sentence, firm)


            else:
              for i in inside_sentences :
                for _sentence in inside_sentences:
                    for firm in firm_list:
                        if firm in _sentence:
                          sentiment = sentiment_analyzer(_sentence, firm)

analyze_sentences(sentences)
from collections import OrderedDict
print(results)
unique_results = list(OrderedDict.fromkeys(tuple(sorted(d.items())) for d in results))
unique_dict_results = [dict(t) for t in unique_results]

from pprint import pprint
pprint(unique_dict_results)
import json
with open('output.json', 'w', encoding='utf-8') as json_file:
    json.dump({"entity_list": firm_list, "results": unique_dict_results}, json_file, ensure_ascii=False, indent=4)

[{'entity': 'Turkcell', 'sentiment': 'olumlu'}, {'entity': 'Vodafone', 'sentiment': 'olumlu'}, {'entity': 'Turkcell', 'sentiment': 'olumlu'}, {'entity': 'Vodafone', 'sentiment': 'olumlu'}]
[{'entity': 'Turkcell', 'sentiment': 'olumlu'},
 {'entity': 'Vodafone', 'sentiment': 'olumlu'}]
